#### Attention score

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [3]:
# 计算每一个token相对于journey的相似度(attention scores)
# 用向量点积作为相似度

query = inputs[1]
att_score = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    att_score[i] = torch.dot(x_i, query)

print(att_score)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [4]:
# softmax 归一化
att_score = torch.softmax(att_score, 0)
print(att_score)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [5]:
# x_1(journey)对其他的token的context vector就是：att_score对其他向量的加权求和
context_vec = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec += att_score[i] * x_i

print(context_vec)

tensor([0.4419, 0.6515, 0.5683])


In [6]:
# 计算出所有的context vector
att_score_all = inputs @ inputs.T

att_score_all = torch.softmax(att_score_all, dim=-1)
print(att_score_all)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [7]:
# context vector(这个上下文向量有什么用)
context_vec_all = att_score_all @ inputs
print(context_vec_all)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


#### 带可更新权重的自注意机制

In [8]:
x_2 = inputs[1] #journey
d_in = inputs.shape[1] # dimension of input
d_out = 2


In [9]:
torch.manual_seed(123)

#可更新的Q,K,V权重矩阵
w_query = torch.nn.Parameter(torch.rand(d_in, d_out))
w_key = torch.nn.Parameter(torch.rand(d_in, d_out))
w_value = torch.nn.Parameter(torch.rand(d_in, d_out))

print(w_query.shape)

torch.Size([3, 2])


In [10]:
# 计算x_2("journey")的Q,K,V值
query_2 = x_2 @ w_query
key_2 = x_2 @ w_key
value_2 = x_2 @ w_value

print(query_2.shape)

torch.Size([2])


In [11]:
# 计算inputs所有token的Q,K,V
query = inputs @ w_query
key = inputs @ w_key
value = inputs @ w_value

print(query)

tensor([[0.2309, 1.0966],
        [0.4306, 1.4551],
        [0.4300, 1.4343],
        [0.2355, 0.7990],
        [0.2983, 0.6565],
        [0.2568, 1.0533]], grad_fn=<MmBackward0>)


In [12]:
# 如何求token1 对 token2 的attention?
# token1的Q dot token2的K
# 例如journey 对 Your的attention
query_2 = query[1]
key_1 = key[0]

attention_2_1 = torch.dot(query_2, key_1)
print(attention_2_1)

tensor(1.2705, grad_fn=<DotBackward0>)


In [13]:
# 求出token2 journey对其他token的attention
attention_2 = query_2 @ key.T
print(attention_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)


In [14]:
# 归一化
d_k = key.shape[-1]
attention_2 = torch.softmax(attention_2 / (d_k ** 0.5), dim=-1) 
print(attention_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)


In [15]:
# 求context vector
context_vec = attention_2 @ value
print(context_vec)

tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)


In [16]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out

        self.w_query = nn.Parameter(torch.rand(d_in, d_out))
        self.w_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.w_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        queries = x @ self.w_query
        keys = x @ self.w_key
        values = x @ self.w_value

        attn_scores = queries @ keys.T
        attn_weight = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)

        context_vec = attn_weight @ values
        return context_vec


In [17]:
torch.manual_seed(123)
selfAttn = SelfAttention_v1(d_in, d_out)
print(selfAttn.forward(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [18]:
#  Listing 3.2 A self-attention class using PyTorch's Linear layers
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [19]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2.forward(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [20]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
d_k = keys.shape[-1]

attn_weight = torch.softmax(attn_scores / d_k ** 0.5, dim = -1)

print(attn_weight)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [21]:
# 应用因果注意力编码
context_length = attn_weight.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))

print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [22]:
attn_weight = attn_weight * mask_simple

#重新归一化
row_sums = attn_weight.sum(dim=-1, keepdim=True)
print(row_sums)
attn_weight = attn_weight / row_sums
print(attn_weight)

tensor([[0.1921],
        [0.3700],
        [0.5357],
        [0.6775],
        [0.8415],
        [1.0000]], grad_fn=<SumBackward1>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [23]:
# 另一种更优的因果注意力编码
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)



tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [24]:
print(inputs.shape)

torch.Size([6, 3])


In [25]:
# 拼batch,模拟多batch输入
batch = torch.stack((inputs, inputs), dim = 0)
print(batch.shape)

torch.Size([2, 6, 3])


$Q = x @ W_q$ 

$K = x @ W_k$

$V = x @ W_v$

$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}}) V$

In [30]:
class CausalAttention(nn.Module):
    def __init__(self, din, dout, context_length, dropout, qkv_bias = False):
        super().__init__()

        self.din = din
        self.dout = dout
        self.dropout = nn.Dropout(dropout)

        self.W_query = nn.Linear(din, dout, bias = qkv_bias)
        self.W_key   = nn.Linear(din, dout, bias = qkv_bias)
        self.W_value = nn.Linear(din, dout, bias = qkv_bias)

        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length),
            diagonal = 1)
        ) #随设备变化自动迁移tensor

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        # self.W_query.weight: dout x din
        # Q = x @ W_query.T
        Q = self.W_query(x) # Q dim: batch_size x num_tokens x dout
        K = self.W_key(x)
        V = self.W_value(x)
        
        # 三维矩阵乘法
        #x(batch_size, num_tokens, din) @ W_query.T(din x dout)
        # W_query广播成 (batch_size x dout x din)
        # for i in range(batch_size):
        #   Q[i] = x[i] @ W_query[i].T

        attn_scores = Q @ K.transpose(1,2) #attn_scores dim: batch_size x num_tokens x num_tokens
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )

        attn_weights = torch.softmax(attn_scores / K.shape[-1] ** 0.5, dim = -1)
        attn_weights = self.dropout(attn_weights)
        
        context_vec = attn_weights @ V #dim: batch_size x num_tokens x dout
        return context_vec

        

In [31]:
torch.manual_seed(123)
context_length = batch.shape[1] #即num tokens

din = 3 # dimension of token embedding
dout = 2 # dimension of context_vector
ca = CausalAttention(din, dout, context_length, 0.)
context_vec = ca(batch)
print(context_vec)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)


In [ ]:
### 多头可以扩阔上下文向量维度，同时并行计算
class MultiAttentionHeaderWrapper(nn.Module):
    def __init__(self, din, dout, context_length, num_headers, drop_out):
        super().__init__()
        
        assert dout % num_headers == 0, "dout should be divisible by num_headers"

        self.headers = nn.ModuleList([CausalAttention(din, dout, context_length, drop_out)
                                      for _ in range(num_headers)])
        

    def forward(self, x):
        return torch.cat([header(x) for header in self.headers], dim = -1)

In [38]:
torch.manual_seed(123)
din, dout = 3, 2
context_length = batch.shape[1]

MAHW = MultiAttentionHeaderWrapper(din, dout, context_length, 2, 0.)

print(MAHW(batch))

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)


In [39]:
# Listing 3.5 An efficient multi-head attention class
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in,  d_out,
                 context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads                        #A
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)                   #B
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
             torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )


    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)                                      #C
        queries = self.W_query(x)                                 #C
        values = self.W_value(x)                                  #C

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)       #D
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)   #D
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim) #D

        keys = keys.transpose(1, 2)                               #E
        queries = queries.transpose(1, 2)                         #E
        values = values.transpose(1, 2)                           #E

        attn_scores = queries @ keys.transpose(2, 3)              #F
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]    #G

        attn_scores.masked_fill_(mask_bool, -torch.inf)           #H

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)     #I

        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)  #J
        context_vec = self.out_proj(context_vec)                  #K
        return context_vec


#A 将投影维度缩小，以匹配期望的输出维度
#B 使用线性层组合头部输出
#C 张量形状：(b, num_tokens, d_out)
#D 我们通过添加 num_heads 维度来隐式地拆分矩阵。然后展开最后一个维度，使其形状从 (b, num_tokens, d_out) 转换为 (b, num_tokens, num_heads, head_dim)
#E 将张量的形状从 (b, num_tokens, num_heads, head_dim) 转置为 (b, num_heads, num_tokens, head_dim)
#F 对每个注意力头进行点积运算
#G 掩码被截断到 token 的数量
#H 使用掩码填充注意力分数
#I 张量形状：（b, num_tokens, n_heads, head_dim）
#J 将多个注意力头的输出结果合并，其中输出维度 self.d_out 等于注意力头数 self.num_heads 与每个头的维度 self.head_dim 的乘积
#K 添加一个可选的线性投影层